In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random

# This if for organize files
import shutil

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

In [3]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

# Organize files

In [4]:
os.chdir('/content/drive/MyDrive/Data/CCSN')
path = os.getcwd()

In [5]:
def mkfile(file):
    if not os.path.exists(file):
        os.makedirs(file)

In [6]:
# Create training and validation set
mkfile('/content/drive/MyDrive/CloudNet/CCSN/train')
mkfile('/content/drive/MyDrive/CloudNet/CCSN/val')
# mkfile('/content/drive/MyDrive/CloudNet/CCSN/test')

In [9]:
# training : validation = 7 : 3
split_rate = 0.3
cloud_class = [cla for cla in os.listdir(path)]
cloud_class

for cla in cloud_class:
    cla_path = path + '/' + cla + '/'
    images = os.listdir(cla_path) # images列表存储了该目录下所有图像的名称
    num = len(images)
    eval_index = random.sample(images, k=int(num * split_rate)) # 对数据进行划分
    train_index = int(0)
    val_index = int(0)
    for index, image in enumerate(images):
        # eval_index 中保存验证集val的图像名称
        if image in eval_index:
            image_path = cla_path + image
            new_path = '/content/drive/MyDrive/CloudNet/CCSN/val'
            shutil.copy(image_path, new_path + '/' + cla + '_' + str(val_index))  # 选中的图像进行复制
            val_index = val_index + 1
        else:
            image_path = cla_path + image
            new_path = '/content/drive/MyDrive/CloudNet/CCSN/train'
            shutil.copy(image_path, new_path + '/' + cla + '_' + str(train_index))  # 选中的图像进行复制
            train_index = train_index + 1
        print("\r[{}] processing [{}/{}]".format(cla, index + 1, num), end="")
    print()

print("processing done!")

[2_Cb] processing [242/242]
[1_As] processing [188/188]
[10_St] processing [202/202]
[0_Ac] processing [221/221]
[5_Cs] processing [287/287]
[7_Cu] processing [182/182]
[9_Sc] processing [340/340]
[6_Ct] processing [200/200]
[3_Cc] processing [268/268]
[8_Ns] processing [274/274]
[4_Ci] processing [139/139]
processing done!


# Transforms

In [7]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    # use for dataset CCSN
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 256)
    # use for dataset CCSN
    transforms.Resize((256, 256)),
    # You may add some transforms here.
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomVerticalFlip(p = 0.5),
    transforms.RandomCrop(size = (256, 256)),
    transforms.ColorJitter(brightness = 0.5, contrast = 0.5, saturation = 0.5, hue = 0.1),
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
    # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])


# Datasets

In [8]:
os.chdir('/content/drive/MyDrive/CloudNet/CCSN')
path = os.getcwd()

In [9]:
class CCSNDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(CCSNDataset).__init__()
        self.path = path
        # print(os.listdir(path))
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path)])
        # print(len(self.files))
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        # print(fname.split("/")[-1])
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        # label = int(fname.split("/")[-1].split("_")[0])
        try:
            # use for dataset CCSN
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label

In [10]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 256, 256]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 256, 256]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 128, 128]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 128, 128]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 64, 64]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 64, 64]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 32, 32]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 32, 32]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 16, 16]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]

            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0)     # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def forward(self, x):
      out = self.cnn(x)
      out = out.view(out.size()[0], -1)
      return self.fc(out)

        
if __name__ == '__main__':
    x = torch.rand([1, 3, 256, 256])
    model = Classifier()
    y = model(x)
    print(model)

Classifier(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [11]:
batch_size = 32
_dataset_dir = '/content/drive/MyDrive/CloudNet/CCSN'
# print(os.path.join(_dataset_dir,"train"))
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = CCSNDataset(os.path.join(_dataset_dir,"train"), tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = CCSNDataset(os.path.join(_dataset_dir,"val"), tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

One /content/drive/MyDrive/CloudNet/CCSN/train sample /content/drive/MyDrive/CloudNet/CCSN/train/0_Ac_0
One /content/drive/MyDrive/CloudNet/CCSN/val sample /content/drive/MyDrive/CloudNet/CCSN/val/0_Ac_0


In [12]:
_exp_name = "sample"

In [ ]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# The number of training epochs and patience.
n_epochs = 300
patience = 50 # If no improvement in 'patience' epochs, early stop

# Initialize a model, and put it on the device specified.
model = Classifier().to(device)

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5) 

# Writer of tensorboard
writer = SummaryWriter()

# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0

for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        # imgs = imgs.half()
        # print(imgs.shape,labels.shape)

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # print(logits)
        # print(labels)

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    writer.add_scalar('Loss/Train', train_loss, epoch)
    writer.add_scalar('Acc/Train', train_acc, epoch)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    writer.add_scalar('Loss/Valid', valid_loss, epoch)
    writer.add_scalar('Acc/Valid', valid_acc, epoch)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # update logs
    if valid_acc > best_acc:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    else:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # save models
    if valid_acc > best_acc:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break

  0%|          | 0/56 [00:00<?, ?it/s]

[ Train | 001/300 ] loss = 2.30603, acc = 0.16778


  0%|          | 0/24 [00:00<?, ?it/s]

[ Valid | 001/300 ] loss = 2.45782, acc = 0.17312
[ Valid | 001/300 ] loss = 2.45782, acc = 0.17312 -> best
Best model found at epoch 0, saving model


  0%|          | 0/56 [00:00<?, ?it/s]

...

  0%|          | 0/56 [00:00<?, ?it/s]

[ Train | 055/300 ] loss = 0.91979, acc = 0.66927


  0%|          | 0/24 [00:00<?, ?it/s]

[ Valid | 055/300 ] loss = 2.24849, acc = 0.37155
[ Valid | 055/300 ] loss = 2.24849, acc = 0.37155


  0%|          | 0/56 [00:00<?, ?it/s]